# UP42 Stereo Satellite Image Ordering 101

Building on a [previous](https://up42.com/blog/tech/data-ordering101) article, this notebook provides a recipe for searching and ordering sterero data from UP42 using the UP42 [Python SDK](https://sdk.up42.com).

 1. Nutshell definition of what stereo/tristereo is.  
 2. Search for stereo/tristereo.
 3. Order stereo/tristereo.
 4. Get a webhook notification for the order delivery.

**N.B.**: Make sure you are using an SDK version **$\ge$ 0.23.0**. This is the version where support for data products was introduced.

## Installing the UP42 Python SDK

The module is called `up42`. Python **>= 3.8** is **required**.

 1. Create a virtual environment.
```bash
 mkvirtualenv --python=$(which python3) up42-py
```
 2. Activate the environment.
```bash
 workon up42-py
```
 3. Install the module.
 ```bash
 pip install up42-py
 ```
 4. Install Jupyter Lab.
```bash
 pip install jupyterlab
 ```
 5. Done.
 
Now we can just import it.

In [1]:
import up42

## Authenticate with project ID and project key
The project API key and the project ID are read from a JSON file. This file has the following structure:
```
{
  "project_id": "<your-project-ID-here>",
  "project_api_key": "<your-api-key-here>"
}
```
To find out how to obtain the project API key and project ID the please refer to the [documentation](https://docs.up42.com/processing-platform/projects#developers)

Reading the credentials from the JSON file `~/.up42/proj_default.json`.

In [2]:
from pathlib import Path

project_config_file = Path.home() / ".up42" / "proj_default.json"
up42.authenticate(cfg_file=project_config_file)

2023-02-17 18:30:00,142 - Got credentials from config file.
2023-02-17 18:30:00,700 - Authentication with UP42 successful!


## Stereo and Tristereo in a nutshell

### Stereo basic concept

**Stereo** in the context of satellite imaging refers to the technique of doing two (or three for tristereo) image captures that are shifted by a certain angle among them over a given scene. The value of this shift determines the level of detail that can be reconstructed from combining the two images. Usually these images are used to create, for example, Digital Elevation Models (DEMs), since the two images when properly combined provide _depth_ information, the relief of the scene being imaged can be determined up to a certain degree of certainty.

### Tristereo basic concept

**Tristereo** adds a third image that is captured **midway** between the two extreme positions of a stereo capture. This corresponds to the satellite being vertically aligned with the scene being imaged. The third photo provides additional detail, including making visible relief details that would be hidden when the photo is taken obliquely, which is the case for stereo.

### Satellite orbits and stereo capture

In this notebook, and in fact for most _modern_ earth observation satellites, these two (ot three) image captures are done in a single orbit pass, be it ascending or descending. This is called **along-track** stereo/tristereo.

## Searching for stero/tristero data

### AOI definition

We now look for existing stereo/tristereo data in the region we are interested in. The AOI is defined via a GeoJSON file in the `examples` directory. Let us use [ipyleaflet](https://ipyleaflet.readthedocs.io/) to visualize it.

We are going to look into to two different AOIs:

 1. To look for SPOT 6/7 stereo image sets.
 2. To look for Pléiades 1A/1B tristereo image sets.

Let us start with the stereo image set.

In [1]:
from ipyleaflet import Map, GeoJSON

In [2]:
# Path to the AOI GeoJSON.
path2aoi = "../examples/portinho_arrabida.geojson"

In [3]:
import json

with open(path2aoi, "r") as f:
    aoi_map = json.load(f)

In [4]:
mymap = Map(center=(38.47064394226866, -8.984506813116552), zoom=14)

# Add the AOI to the map. First style it and then add it.
aoi_layer = GeoJSON(
    data=aoi_map,
    style={"opacity": 1, "dashArray": "9", "fillOpacity": 0.5, "weight": 1},
    hover_style={"color": "yellow", "dashArray": "0", "fillOpacity": 0.5},
)
mymap.add_layer(aoi_layer)
mymap

Map(center=[38.47064394226866, -8.984506813116552], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [7]:
type(aoi_layer)

ipyleaflet.leaflet.GeoJSON

### Searching for data in the catalog
We need to instantiate a catalog object and perform a search.

In [10]:
catalog = up42.initialize_catalog()

#### Getting the display product ID

Before performing the search we need to select the data product we are interested in. In this case we are interested in a display product of a Pléiades image. The data product we want is not necessary for the search itself, but we do it now so that the ordering flow stays simple. Since we do the data product selection at the outset and not after performing the search. This reproduces the usual actions of having the product specified before hand and then performing the search.

In [11]:
products = catalog.get_data_products(basic=True)
products

{'Near Space Labs - 30cm': {'collection': 'nsl-30cm',
  'host': 'nearspacelabs',
  'data_products': {'Display Full Scene': '7131f727-cdbb-46d4-a1e7-77ac38b7bd02'}},
 'Capella Space GEC': {'collection': 'capella-gec',
  'host': 'capellaspace',
  'data_products': {'Full Scene': '96072809-d820-4cf9-86dd-d3bff3337c35'}},
 'Capella Space GEO': {'collection': 'capella-geo',
  'host': 'capellaspace',
  'data_products': {'Full Scene': 'd66facaa-533f-49a2-849a-c2910ac9dd31'}},
 'Capella Space SICD': {'collection': 'capella-sicd',
  'host': 'capellaspace',
  'data_products': {'Full Scene': '8b0aed07-c565-4bf9-b719-401e692de4a6'}},
 'Capella Space SLC': {'collection': 'capella-slc',
  'host': 'capellaspace',
  'data_products': {'Full Scene': '1f2b0d7f-d3e2-4b3d-96b7-e7c184df7952'}},
 'Pléiades Neo': {'collection': 'pneo',
  'host': 'oneatlas',
  'data_products': {'Analytic': '6f722fb5-0323-4c97-9b1e-22870325b8db',
   'Display': '17745de8-6e7d-4751-99cd-3f8e9e9d290e'}},
 'SPOT 6/7': {'collection':

We have the list of all **available** collections and the corresponding products. In our case we want SPOT 6/7.

In [12]:
spot_product = next(filter(lambda e: e[0].lower().startswith("spot"), products.items()))
spot_product

('SPOT 6/7',
 {'collection': 'spot',
  'host': 'oneatlas',
  'data_products': {'Analytic': 'b1f8c48e-d16b-44c4-a1bb-5e8a24892e69',
   'Display': 'acc3f9a4-b622-49c1-b1e1-c762aa3e7e13'}})

Specifically we want a display product. We need to get the corresponding product ID.

In [13]:
spot_product_id = spot_product[1]["data_products"]["Display"]
spot_product_id

'acc3f9a4-b622-49c1-b1e1-c762aa3e7e13'

#### Performing the search


We are going to look for data from September 1st, 2021 until now. We set the upper date range to be Dec 31st 2022.
We need the collection name.

In [14]:
spot_coll_name = spot_product[1]["collection"]
spot_coll_name

'spot'

The search parameters are defined below we are asking for up to **50** results sorted by descending acquisition date.

In [15]:
spot_search_params = catalog.construct_search_parameters(
    geometry=aoi_map,
    start_date="2021-09-01",
    end_date="2022-12-31",
    collections=[spot_coll_name],
    max_cloudcover=10,
    sortby="acquisitionDate",
    ascending=False,
    limit=50,
)

We ask for the results to be returned as `FeatureCollection` in a Python dict the resulting found features are returned in a list. This is more convenient for later filtering of the stereo pairs/tristereo triples.

In [16]:
spot_search_results = catalog.search(spot_search_params, as_dataframe=False)

2023-02-16 14:37:49,846 - Searching catalog with search_parameters: {'datetime': '2021-09-01T00:00:00Z/2022-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((-8.970508575439453, 38.48382914339068), (-8.98106575012207, 38.48141037533113), (-8.989777565002441, 38.47637101450613), (-8.992223739624023, 38.47015531764655), (-8.991966247558594, 38.46561920067743), (-8.990163803100586, 38.465518394837304), (-8.990421295166016, 38.46709767014416), (-8.986902236938477, 38.46978571879323), (-8.982696533203125, 38.47381760389001), (-8.984713554382324, 38.47600144751436), (-8.981409072875977, 38.47905871745382), (-8.97707462310791, 38.48026815108751), (-8.971881866455078, 38.48016736572641), (-8.96934986114502, 38.480604101273386), (-8.969564437866211, 38.481746320193345), (-8.968620300292969, 38.48208226348981), (-8.969006538391113, 38.48288852101284), (-8.970508575439453, 38.48382914339068)),)}, 'limit': 50, 'collections': ['spot'], 'query': {'cloudCoverage': {'lte': 10}}, 's

In [17]:
# N.B: This is an ugly hack that needs to be replaced by creating a proper local Python package for the Notebook helpers.
import sys
sys.path.append(str((Path.cwd() / ".." / "..").resolve()))

In [18]:
from pynb_helpers import stereo

Let us look for the stereo pairs in the above search results.

In [19]:
spot_stereo_results = stereo.select_stereo(spot_search_results["features"])
spot_stereo_results

[({'id': '3',
   'type': 'Feature',
   'properties': {'id': '78d9e0e4-0f4a-49f8-adc5-02895693bc7c',
    'constellation': 'spot',
    'collection': 'spot',
    'providerName': 'oneatlas',
    'up42:usageType': ['DATA'],
    'providerProperties': {'qualified': False,
     'sourceIdentifier': 'DS_SPOT6_202110131059016_FR1_FR1_FR1_FR1_W009N38_01140',
     'acquisitionStation': 'FR1',
     'archivingCenter': 'FR1',
     'lastUpdateDate': '2021-10-27T14:30:11Z',
     'processingDate': '2021-10-13T11:27:28.544Z',
     'parentIdentifier': 'DS_SPOT6_202110131059016_FR1_FR1_FR1_FR1_W009N38_01140',
     'azimuthAngle': 180.02822144097493,
     'workspaceTitle': 'Public',
     'spectralRange': 'VISIBLE',
     'productCategory': 'image',
     'activityId': '064b9a3b-e95d-4c52-91c9-f4e31a9acce9',
     'processingLevel': 'ALBUM',
     'acquisitionIdentifier': None,
     'productionStatus': 'ARCHIVED',
     'sensorType': 'OPTICAL',
     'geometryCentroid': [-8.858839421206234, 38.488066997121585],
   

#### Get the stereo image IDs for ordering

We need to get the image IDs for placing the order. We use the *convenience* function `get_stereo_image_ids`.

In [20]:
stereo.get_stereo_image_ids(spot_stereo_results)

['78d9e0e4-0f4a-49f8-adc5-02895693bc7c',
 'b269a05e-c92b-4480-af7c-57139eff2acf',
 'e72c9441-4329-4c4d-ac22-837369c31813',
 '5fc38a28-89a1-43a7-a63a-96e6a5ee0a87',
 '4148e24f-25ee-4e18-96df-912aff6c3221',
 'ff448805-c3c6-4e02-8bae-1daa198b2325']

We have now all we need to go ahead and order the data.

### Looking for tristereo data

After having found stereo pairs, let us now look for tristereo triples. We define another AOI to look for tristereo data.

In [21]:
# Path to the AOI GeoJSON.
path2aoi2 = "../examples/figueira_foz.geojson"

In [22]:
import json

with open(path2aoi2, "r") as f:
    aoi_map2 = json.load(f)

In [23]:
mymap2 = Map(center=(40.157387868535764, -8.857583004391348), zoom=14)

# Add the AOI to the map. First style it and then add it.
aoi_layer = GeoJSON(
    data=aoi_map2,
    style={"opacity": 1, "dashArray": "9", "fillOpacity": 0.5, "weight": 1},
    hover_style={"color": "yellow", "dashArray": "0", "fillOpacity": 0.5},
)
mymap2.add_layer(aoi_layer)
mymap2

Map(center=[40.157387868535764, -8.857583004391348], controls=(ZoomControl(options=['position', 'zoom_in_text'…

Let us get the product ID for the display product.

In [26]:
pleiades_product = next(
    filter(lambda e: e[0].lower().endswith("ades"), products.items())
)
pleiades_product

('Pléiades',
 {'collection': 'phr',
  'host': 'oneatlas',
  'data_products': {'Analytic': '4f1b2f62-98df-4c74-81f4-5dce45deee99',
   'Display': '647780db-5a06-4b61-b525-577a8b68bb54'}})

In [27]:
pleiades_product_id = pleiades_product[1]["data_products"]["Display"]
pleiades_product_id

'647780db-5a06-4b61-b525-577a8b68bb54'

And the collection name.

In [28]:
pleiades_coll_name = pleiades_product[1]["collection"]
pleiades_coll_name

'phr'

Let us now perform the search.

In [29]:
pleiades_search_params = catalog.construct_search_parameters(
    geometry=aoi_map2,
    start_date="2021-09-01",
    end_date="2022-12-31",
    collections=[pleiades_coll_name],
    max_cloudcover=10,
    sortby="acquisitionDate",
    ascending=False,
    limit=50,
)

In [30]:
pleiades_search_results = catalog.search(pleiades_search_params, as_dataframe=False)

2023-02-16 14:41:32,641 - Searching catalog with search_parameters: {'datetime': '2021-09-01T00:00:00Z/2022-12-31T23:59:59Z', 'intersects': {'type': 'Polygon', 'coordinates': (((-8.884849548339844, 40.16641243354131), (-8.884334564208984, 40.16601889477649), (-8.865280151367188, 40.12232191122354), (-8.798847198486328, 40.13098480788124), (-8.801078796386719, 40.15604848507155), (-8.831291198730469, 40.17480738367364), (-8.884849548339844, 40.16641243354131)),)}, 'limit': 50, 'collections': ['phr'], 'query': {'cloudCoverage': {'lte': 10}}, 'sortby': [{'field': 'properties.acquisitionDate', 'direction': 'desc'}]}
2023-02-16 14:41:45,441 - 11 results returned.


In [31]:
pleiades_tristereo_results = stereo.select_tristereo(
    pleiades_search_results["features"]
)
pleiades_tristereo_results

[({'id': '6',
   'type': 'Feature',
   'properties': {'id': 'bc84ea63-f76b-47df-a868-373dacef9a17',
    'constellation': 'phr',
    'collection': 'phr',
    'providerName': 'oneatlas',
    'up42:usageType': ['DATA'],
    'providerProperties': {'commercialReference': 'SO21065601',
     'qualified': False,
     'sourceIdentifier': 'DS_PHR1A_202110071120575_FR1_PX_W009N40_0102_03614',
     'acquisitionStation': 'FR1',
     'archivingCenter': 'FR1',
     'lastUpdateDate': '2021-10-08T01:29:28.297605581Z',
     'processingDate': '2021-10-08T00:15:24.337',
     'parentIdentifier': 'DS_PHR1A_202110071121345_FR1_PX_W009N40_0102_03618',
     'azimuthAngle': 179.9662671455482,
     'workspaceTitle': 'Public',
     'spectralRange': 'VISIBLE',
     'productCategory': 'image',
     'activityId': '48bc0d1c-8c89-4174-9bb2-5d14d6a30cd8',
     'processingLevel': 'SENSOR',
     'acquisitionIdentifier': None,
     'customerReference': 'C301000',
     'productionStatus': 'IN_CLOUD',
     'sensorType': 'OP

We need to get the image IDs for placing the order. We use the *convenience* function `get_tristereo_image_ids`.

In [32]:
stereo.get_tristereo_image_ids(pleiades_tristereo_results)

['bc84ea63-f76b-47df-a868-373dacef9a17',
 '0a0a6a09-0ef5-4799-ab05-ecb4abab9c92',
 '997a983c-3de1-4480-b3ee-be7df7b108f9']

## Placing the orders

For placing the orders and getting notified via a webhook when the orders get delivered we refer you to the recipe described in the blog post [data ordering 101](https://up42.com/blog/tech/data-ordering101) and the accompanying [Jupyter notebook](https://github.com/up42/data-recipes/blob/master/stereo-ordering101/notebooks/stereo_ordering101.ipynb).

## Obtaining primary products

Currently it is not possible to obtain primary products via API from Airbus. If you want to create a DEM then we can obtain these products for you. 

 1. From the process described in the notebook you can get the information you need to place the order.
 2. Send your request via email to [ordering@up42.com](mailto:ordering@up42.com).